In [1]:
from math import floor

In [2]:
def parse_opcode(opcode):
    if opcode >= 100: # two-digit opcode
        op    = opcode % 100
        mode1 = floor(opcode % 1000 / 100)
        mode2 = floor(opcode % 10000 / 1000)
        mode3 = floor(opcode % 100000 / 10000)
    else:
        op = opcode
        mode1, mode2, mode3 = 0, 0, 0 # position mode
    return op, mode1, mode2, mode3

In [12]:
parse_opcode(204)

(4, 2, 0, 0)

In [14]:
opcode, mode1, mode2, mode3 = parse_opcode(109)

In [15]:
opcode, mode1, mode2, mode3

(9, 1, 0, 0)

In [30]:
parse_opcode(11109)

(9, 1, 1, 1)

In [31]:
parse_opcode(1001)

(1, 0, 1, 0)

In [105]:
def fetch_value(data, arg, mode, base):
    if mode == 0: # position
        return data[arg]
    elif mode == 1: # immediate
        return arg
    elif mode == 2: # relative
        return data[base+arg]

In [127]:
fetch_value([3,0,4,0,99], -7, 2, 7)

3

In [135]:
def store_value(data, arg, val, mode, base):
    if mode == 0: # position
        data[arg] = val
    elif mode == 1: # immediate
        raise ValueError("store_value cannot be immediate [arg=%d, val%d, mode=%d, base=%d]" \
                        % (arg, val, mode, base))
    elif mode == 2: # relative
        data[base+arg] = val

In [148]:
def interpret(data, pos, base):
    opcode, mode1, mode2, mode3 = parse_opcode(data[pos])
    if opcode == 1: # add
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]
        
        val1 = fetch_value(data, arg1, mode1, base)
        val2 = fetch_value(data, arg2, mode2, base)
        
        store_value(data, arg3, val1 + val2, mode3, base)
        return interpret(data, pos + 4, base)
    elif opcode == 2: # multiply
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]
        
        val1 = fetch_value(data, arg1, mode1, base)
        val2 = fetch_value(data, arg2, mode2, base)
        
        store_value(data, arg3, val1 * val2, mode3, base)
        return interpret(data, pos + 4, base)
    elif opcode == 3: # input
        arg1 = data[pos+1]
        
        val1 = int(input())

        store_value(data, arg1, val1, mode1, base)
        return interpret(data, pos + 2, base)
    elif opcode == 4: # output
        arg1 = data[pos+1]
        val1 = fetch_value(data, arg1, mode1, base)
        print(val1)
        return interpret(data, pos + 2, base)
    elif opcode == 5: # jump-if-true
        arg1 = data[pos+1]
        arg2 = data[pos+2]

        val1 = fetch_value(data, arg1, mode1, base)
        val2 = fetch_value(data, arg2, mode2, base)
        
        if val1 != 0:
            pos = val2
            return interpret(data, pos, base)
    
        return interpret(data, pos+3, base)
    elif opcode == 6: # jump-if-false
        arg1 = data[pos+1]
        arg2 = data[pos+2]

        val1 = fetch_value(data, arg1, mode1, base)
        val2 = fetch_value(data, arg2, mode2, base)
        
        if val1 == 0:
            pos = val2
            return interpret(data, pos, base)
        
        return interpret(data, pos+3, base)
    elif opcode == 7: # less than
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]

        val1 = fetch_value(data, arg1, mode1, base)
        val2 = fetch_value(data, arg2, mode2, base)
        
        if val1 < val2:
            store_value(data, arg3, 1, mode3, base)
        else:
            store_value(data, arg3, 0, mode3, base)
        
        return interpret(data, pos + 4, base)
    elif opcode == 8: # equals
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]

        val1 = fetch_value(data, arg1, mode1, base)
        val2 = fetch_value(data, arg2, mode2, base)
        
        if val1 == val2:
            store_value(data, arg3, 1, mode3, base)
        else:
            store_value(data, arg3, 0, mode3, base)
        
        return interpret(data, pos + 4, base)
    elif opcode == 9: # adjust relative base
        arg1 = data[pos+1]
        
        base = base + fetch_value(data, arg1, mode1, base)
        
        return interpret(data, pos + 2, base)
    elif opcode == 99: # end
        return data

In [115]:
# ¯\_(ツ)_/¯
def growProg(ex2prog, amount):
    for i in range(0, amount):
        ex2prog.append(0)

In [116]:
ex1prog = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]

In [117]:
len(ex1prog)

16

In [118]:
growProg(ex1prog, 100)

In [119]:
len(ex1prog)

116

In [120]:
outprog = interpret(ex1prog, 0, 0)

109
1
204
-1
1001
100
1
100
1008
100
16
101
1006
101
0
99


In [121]:
ex2prog = [1102,34915192,34915192,7,4,7,99,0]

In [122]:
outprog = interpret(ex2prog, 0, 0)

1219070632396864


In [123]:
len("1219070632396864")

16

In [47]:
ex3prog = [104,1125899906842624,99]

In [49]:
outprog = interpret(ex3prog, 0, 0)

1125899906842624


In [52]:
def loadInput():
    with open("input") as infile:
        data = infile.read()
    data = data.rstrip()
    inputData = data.split(",")
    inputData = list(map(int, inputData))
    return inputData

In [149]:
inputData = loadInput()

In [150]:
growProg(inputData, 10000)

In [151]:
outprog = interpret(inputData, 0, 0)

1
3454977209


In [158]:
def interpretIter(data):
    running = True
    pos = 0
    base = 0

    while running:
        opcode, mode1, mode2, mode3 = parse_opcode(data[pos])
        if opcode == 1: # add
            arg1 = data[pos+1]
            arg2 = data[pos+2]
            arg3 = data[pos+3]
            
            val1 = fetch_value(data, arg1, mode1, base)
            val2 = fetch_value(data, arg2, mode2, base)
            
            store_value(data, arg3, val1 + val2, mode3, base)
            pos = pos + 4
        elif opcode == 2: # multiply
            arg1 = data[pos+1]
            arg2 = data[pos+2]
            arg3 = data[pos+3]
            
            val1 = fetch_value(data, arg1, mode1, base)
            val2 = fetch_value(data, arg2, mode2, base)
            
            store_value(data, arg3, val1 * val2, mode3, base)
            pos = pos + 4
        elif opcode == 3: # input
            arg1 = data[pos+1]
            
            val1 = int(input())

            store_value(data, arg1, val1, mode1, base)
            pos = pos + 2
        elif opcode == 4: # output
            arg1 = data[pos+1]
            val1 = fetch_value(data, arg1, mode1, base)
            print(val1)
            pos = pos + 2
        elif opcode == 5: # jump-if-true
            arg1 = data[pos+1]
            arg2 = data[pos+2]

            val1 = fetch_value(data, arg1, mode1, base)
            val2 = fetch_value(data, arg2, mode2, base)
            
            if val1 != 0:
                pos = val2
            else:
                pos = pos + 3
        elif opcode == 6: # jump-if-false
            arg1 = data[pos+1]
            arg2 = data[pos+2]

            val1 = fetch_value(data, arg1, mode1, base)
            val2 = fetch_value(data, arg2, mode2, base)
            
            if val1 == 0:
                pos = val2
            else:
                pos = pos + 3
        elif opcode == 7: # less than
            arg1 = data[pos+1]
            arg2 = data[pos+2]
            arg3 = data[pos+3]

            val1 = fetch_value(data, arg1, mode1, base)
            val2 = fetch_value(data, arg2, mode2, base)
            
            if val1 < val2:
                store_value(data, arg3, 1, mode3, base)
            else:
                store_value(data, arg3, 0, mode3, base)
            
            pos = pos + 4
        elif opcode == 8: # equals
            arg1 = data[pos+1]
            arg2 = data[pos+2]
            arg3 = data[pos+3]

            val1 = fetch_value(data, arg1, mode1, base)
            val2 = fetch_value(data, arg2, mode2, base)
            
            if val1 == val2:
                store_value(data, arg3, 1, mode3, base)
            else:
                store_value(data, arg3, 0, mode3, base)
            
            pos = pos + 4
        elif opcode == 9: # adjust relative base
            arg1 = data[pos+1]
            
            base = base + fetch_value(data, arg1, mode1, base)
            
            pos = pos + 2
        elif opcode == 99: # end
            running = False

In [159]:
inputData = loadInput()

In [160]:
growProg(inputData, 10000)

In [163]:
outprog = interpretIter(inputData)

2
50120
